In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

In [ ]:
sys.path.append("../crazyflie-audio/python/")
from bin_selection import select_frequencies
from crazyflie_description_py.parameters import N_BUFFER, FS, FFTSIZE

DURATION_PER_FREQ = 0.5
DURATION_SEC = DURATION_PER_FREQ * FFTSIZE
print(DURATION_SEC)


def buzzer_signal(source):
    from signals import generate_signal

    kwargs = dict(duration_sec=DURATION_SEC, max_dB=-10, Fs=FS,)
    frequencies = np.fft.rfftfreq(N_BUFFER, 1 / FS)
    if source == "mono4125":
        return generate_signal(signal_type="mono", frequency_hz=4125, **kwargs)
    elif source == "random":
        return generate_signal(signal_type="random", **kwargs)
    elif source == "multi":
        phase = np.random.uniform(0, 2 * np.pi)
        signal = generate_signal(
            signal_type="mono",
            frequency_hz=frequencies[1],
            **kwargs,
            phase_offset=phase,
        )
        for f in frequencies[2:]:
            phase = np.random.uniform(0, 2 * np.pi)
            signal += generate_signal(
                signal_type="mono", frequency_hz=f, **kwargs, phase_offset=phase
            )
        return signal
    elif source == "sweep":
        # duration of each frequency bin
        min_freq = 100
        max_freq = 4400

        bins = select_frequencies(
            n_buffer=N_BUFFER, fs=FS, min_freq=min_freq, max_freq=max_freq
        )

        duration_all = len(bins) * duration_per_freq
        signal = np.zeros(int(FS * duration_all))
        idx = 0
        for f in frequencies[bins]:

            kwargs["duration_sec"] = duration_per_freq
            mono = generate_signal(signal_type="mono", frequency_hz=f, **kwargs)
            signal[idx : idx + len(mono)] = mono
            idx += len(mono)
        return signal
    else:
        raise ValueError(source)


signals = {}
for key in ["mono4125", "random", "multi", "sweep"]:
    try:
        signal = pd.read_pickle(f"results/{key}.pk")
        print(f"read {key}")
    except:
        signal = buzzer_signal(key)
        pd.to_pickle(signal, f"results/{key}.pk")
        print(f"saved {key}")
    signals[key] = signal

In [ ]:
def matrix_slices(df_matrix, frequencies, distances, saveas="", ymin=None, ymax=None):
    from ipywidgets import (
        interact,
        interactive,
        fixed,
        interact_manual,
        FloatSlider,
        IntSlider,
    )
    import ipywidgets as widgets

    def update_figure(slice_f, slice_d):
        fig, axs = plt.subplots(1, 2)
        fig.set_size_inches(10, 3)
        # axs[0].semilogy(distances, df_matrix[slice_f, :])
        axs[0].plot(distances, df_matrix[slice_f, :])
        axs[0].set_title(f"frequency = {frequencies[slice_f]:.0f} Hz")
        axs[0].set_xlabel("distance [cm]")
        axs[0].set_ylim(ymin, ymax)

        # axs[1].semilogy(frequencies, df_matrix[:, slice_d])
        axs[1].plot(frequencies, df_matrix[:, slice_d])
        axs[1].set_title(f"distance = {distances[slice_d]:.0f} cm")
        axs[1].set_xlabel("frequency [Hz]")
        axs[1].set_ylim(ymin, ymax)
        if saveas != "":
            fig.savefig(saveas, bbox_inches="tight")
        plt.show()

    interactive_plot = interact(
        update_figure,
        slice_f=IntSlider(
            200, min=0, max=len(frequencies) - 1, continuous_update=False
        ),
        slice_d=IntSlider(50, min=0, max=len(distances) - 1, continuous_update=False),
    )

In [ ]:
from simulation import generate_room
angles = [0, 20, 90]
fig, axs = plt.subplots(1, len(angles), squeeze=False)
fig.set_size_inches(5*len(angles), 5)
for ax, angle in zip(axs.flatten(), angles):
    room = generate_room(distance_cm=0, yaw_deg=angle, ax=ax)
    ax.set_title(f'yaw{angle}')
    ax.legend(loc="lower left")

# Distance-frequency matrix

In [ ]:
from simulation import get_df_theory_simple, get_deltas_from_global
fig, ax = plt.subplots()
for i in range(4):
    deltas = get_deltas_from_global(20, 0, i, ax)
ax.axvline(10, color='black', label='wall')
ax.legend()
ax.axis('equal')

In [ ]:
from simulation import get_dist_slice_pyroom, get_dist_slice_theory

yaw_deg = 20
distances_cm = np.arange(20)
frequency = 1000

Hs = get_dist_slice_pyroom(frequency, distances_cm=distances_cm, yaw_deg=yaw_deg)
Hs_theo = get_dist_slice_theory(frequency, distances_cm=distances_cm, yaw_deg=yaw_deg)
fig, ax = plt.subplots()
for i in range(Hs.shape[1]):
    ax.semilogy(distances_cm, Hs[:, i], label=f"mic{i}", ls="-", color=f"C{i}")
    ax.semilogy(distances_cm, Hs_theo[:, i], ls=":", color=f"C{i}")
ax.legend()

In [ ]:
from simulation import get_freq_slice_pyroom
import progressbar

mic_idx = 1
distances = np.linspace(0, 60, 100)
frequencies = np.fft.rfftfreq(N_BUFFER, 1 / FS)

fname = "results/df_matrix_pyroom.pkl"
try:
    series_all = pd.read_pickle(fname)
    print("read", fname)
    np.testing.assert_allclose(series_all.distances, distances)
    np.testing.assert_allclose(series_all.frequencies, frequencies)
    df_matrix = series_all.df_matrix
except Exception as e:
    print(e)
    df_matrix = np.zeros((len(frequencies), len(distances)))

    signal = signals["multi"]
    with progressbar.ProgressBar(max_value=len(distances) - 1) as bar:
        for j, distance_cm in enumerate(distances):
            slice_mics = get_freq_slice_pyroom(frequencies, distance_cm, signal=signal)
            df_matrix[:, j] = slice_mics[mic_idx]
            bar.update(j)
    series = pd.Series(
        {"df_matrix": df_matrix, "distances": distances, "frequencies": frequencies}
    )
    pd.to_pickle(series, fname)
    print("saved as", fname)

In [ ]:
min_freq = 100
max_freq = 5000
min_dist = 1
max_dist = 50
freq_start = int(min_freq / max(frequencies) * len(frequencies))
freq_end = int(max_freq / max(frequencies) * len(frequencies))
dist_start = int(min_dist / max(distances) * len(distances))
dist_end = int(max_dist / max(distances) * len(distances))

dist = distances[dist_start:dist_end]
freq = frequencies[freq_start:freq_end]

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
ax.pcolorfast(dist, freq, np.log10(df_matrix[freq_start:freq_end, dist_start:dist_end]))
plt.xlabel("distance [cm]")
plt.ylabel("frequency [Hz]")
fname = "plots/distance-frequency-matrix.pdf"
fig.savefig(fname, bbox_inches="tight")
fname = "plots/distance-frequency-matrix-slice.pdf"
matrix_slices(
    df_matrix[freq_start:freq_end, dist_start:dist_end],
    freq,
    dist,
    saveas=fname,
    ymin=8,
    ymax=20,
)

In [ ]:
from simulation import get_freq_slice_theory
df_matrix_theo = np.empty((len(frequencies), len(distances)))
for j, distance_cm in enumerate(distances):
    slice_mics = get_freq_slice_theory(frequencies, distance_cm)
    df_matrix_theo[:, j] = slice_mics[:, mic_idx]
    # H = generate_rir(frequencies, distance_cm=distance_cm, single_mic=False)[mic_idx] #
    # df_matrix_theo[:, j] = np.abs(H)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
ax.pcolorfast(
    dist, freq, np.log10(df_matrix_theo[freq_start:freq_end, dist_start:dist_end])
)
matrix_slices(
    df_matrix_theo[freq_start:freq_end, dist_start:dist_end],
    freq,
    dist,
    ymin=0.0,
    ymax=3.0,
)

In [ ]:
from constants import SPEED_OF_SOUND

# matrix = df_matrix
matrix = df_matrix_theo

# frequencies_slices = [671, 3125, 4156, 8000]
frequencies_slices = [1750, 2375, 3125, 3875]
# frequencies_slices = [703 * (i + 1) for i in range(4)]
# frequencies_slices = [359,  703, 1406]

max_distance = 60
fig, axs = plt.subplots(len(frequencies_slices), sharey=True, sharex=True)
fig.suptitle("distance slices", y=0.95)
fig.set_size_inches(10, 5)
for i, f in enumerate(frequencies_slices):
    idx = np.argmin(np.abs(frequencies - f))
    slice_ = matrix[idx, distances < max_distance]
    # slice_ -= np.mean(slice_)

    d_constr = SPEED_OF_SOUND / (2 * f) * 100
    axs[i].plot(
        distances[distances < max_distance],
        slice_,
        label=f"{f}Hz, period={d_constr:.0f}cm",
    )
    axs[i].set_ylabel("response")
    axs[i].legend(loc="upper right")

    # peak from constr. interference is expected when the wavelength equals k * distance/2
    # wavelength_constr = c / f_constr = distance / 2
    num_peaks = (distances[-1] - distances[0]) // d_constr + 1
    [axs[i].axvline(x=d, color="C1") for d in np.arange(num_peaks) * d_constr]

    fft = np.fft.rfft(slice_)[1:]
    df = np.mean(distances[1:] - distances[:-1])
    freq = np.fft.rfftfreq(len(slice_), df)[1:]

    plt.figure()
    plt.plot(1 / freq, np.abs(fft))
    plt.axvline(d_constr, color="C1")
axs[-1].set_xlabel("distance [cm]")
fig.savefig("plots/distance-frequency-matrix-algorithm.pdf", bbox_inches="tight")

### vertical frequency slice

In [ ]:
from wall_detector import get_probability_cost, get_probability_fft
from simulation import get_deltas_from_global

distance_slices = [5, 10, 20, 30, 40, 50]

window = "hann"  # None #'hann' #('tukey', 0.2) #None
max_frequency = 5000
min_frequency = 1000
n_freqs = 32
freq_indices = np.where((frequencies < max_frequency) & (frequencies > min_frequency))[
    0
]
sparse_indices = np.linspace(freq_indices[0], freq_indices[-1], n_freqs).astype(int)

freq_cont = frequencies[freq_indices]
freq_sparse = frequencies[sparse_indices]

matrix = df_matrix_theo

distances_grid = np.linspace(0, 70, 70)
distances_real = [
    get_deltas_from_global(yaw_deg=0, distances_cm=[d], mic_idx=mic_idx)[0]/2 * 100
    for d in distances_grid
]

fig, axs = plt.subplots(len(distance_slices), sharey=True, sharex=True)
fig.suptitle("frequency slices")
fig.set_size_inches(10, 10)
for i, d in enumerate(distance_slices):
    idx = np.argmin(np.abs(distances - d))
    slice_cont = matrix[freq_indices, idx] ** 2
    slice_sparse = matrix[sparse_indices, idx] ** 2

    d_cont, p_fft_cont = get_probability_fft(slice_cont, freq_cont, window=window)
    d_sparse, p_fft_sparse = get_probability_fft(
        slice_sparse, freq_sparse, window=window
    )

    # fig, ax = plt.subplots()
    # p_cost_cont = get_probability_cost(slice_cont, freq_cont, distances_grid, ax=ax)
    p_cost_cont = get_probability_cost(slice_cont, freq_cont, distances_grid)
    p_cost_sparse = get_probability_cost(slice_sparse, freq_sparse, distances_grid)

    # peak from constr. interference is expected when the wavelength equals k * distance/2
    delta = get_deltas_from_global(yaw_deg=0, distances_cm=[d], mic_idx=mic_idx)[0] * 100

    axs[i].set_ylabel("response")
    axs[i].plot(freq_cont, slice_cont, label=f"{d}cm", color="C0")
    axs[i].scatter(freq_sparse, slice_sparse, color="C0")
    axs[i].legend(loc="upper right")

    plt.figure()
    plt.plot(d_sparse, p_fft_sparse, marker="o", label="FFT")
    plt.plot(distances_real, p_cost_sparse, marker="o", label="optimization")
    plt.axvline(delta / 2, color="black", label="real distance")
    plt.title(f"{d}cm")
    plt.xlim(0, 70)
    plt.ylim(1e-4, 1)
    plt.legend()
    plt.yscale("log")
axs[i].set_xlabel("frequency [Hz]")

### non-vertical frequency slice

In [ ]:
nominal_distance = 10
nominal_yaw = 0
relative_distances = np.random.normal(0, 1, len(freq_sparse))
absolute_yaws = np.random.normal(0, 1, len(freq_sparse))

deltas = get_deltas_from_global(yaw_deg=nominal_yaw, distances_cm=distances, mic_idx=mic_idx)
nominal_delta = get_deltas_from_global(nominal_yaw, nominal_distance, mic_idx)

deltas_noisy = get_deltas_from_global(nominal_yaw+absolute_yaws, 
                                nominal_distance+relative_distances, 
                                mic_idx)

fig, ax = plt.subplots()
ax.pcolorfast(deltas, frequencies, matrix[:-1, :-1])
ax.axvline(nominal_delta, color='C1')
ax.scatter(deltas_noisy, freq_sparse, color='C1')
ax.set_ylim(min(freq_sparse), max(freq_sparse))

df_vert = get_df_theory_simple(nominal_delta, freq_sparse) 
slice_f_vert = df_vert.flatten()

df = get_df_theory_simple(deltas_noisy, freq_sparse) 
slice_f = np.diag(df)

plt.figure()
plt.plot(freq_sparse, slice_f_vert, label='vertical')
plt.plot(freq_sparse, slice_f, label='noisy')
plt.legend()

probs_rel = get_probability_cost(slice_f, freq_sparse, distances_grid, 
                                 relative_ds=relative_distances, 
                                 absolute_yaws=absolute_yaws)
probs = get_probability_cost(slice_f, freq_sparse, distances_grid)
plt.figure()
plt.plot(distances_grid, probs, label='ignore mvmt')
plt.plot(distances_grid, probs_rel, label='account for mvmt')
plt.yscale('log')
plt.legend()

# Noise study

In [ ]:
from simulation import get_delta
d0 = 5  # [cm]
d_arr = np.arange(5, 50, step=5)
beta_arr = np.linspace(-np.pi, np.pi, 361)
delta_matrix = get_delta(d_arr[None, :], beta_arr[:, None], d0)

plt.figure()
for j, d in enumerate(d_arr):
    plt.plot(beta_arr * 180 / np.pi, delta_matrix[:, j], label=f"{d}cm")
plt.xlabel("beta [deg]")
plt.ylabel("delta [cm]")
plt.legend(bbox_to_anchor=[1.0, 1.0], loc="upper left", title="orthogonal distance")
plt.grid()

In [ ]:
from plotting_tools import save_fig
from simulation import get_orthogonal_distance

d0 = 5
delta_arr = np.arange(10, 100, step=10)
beta_arr = np.linspace(-np.pi, np.pi, 361)
d_matrix = get_orthogonal_distance(delta_arr, beta_arr, d0)

fig = plt.figure()
fig.set_size_inches(5, 5)
for j, d in enumerate(delta_arr):
    plt.plot(beta_arr * 180 / np.pi, d_matrix[:, j], label=f"{d}cm")
plt.xlabel("wall angle [deg]")
plt.ylabel("orthogonal distance [cm]")
plt.grid(which="both")
plt.legend(bbox_to_anchor=[1.0, 1.0], loc="upper left", title="delta")
save_fig(fig, "plots/theory_distances.png")